In [1]:
import os
import pandas as pd
path = os.path.dirname(os.path.realpath('__file__'))
print(path)
df = pd.read_csv(path + '\\EEG\\user_3\\user_3_loc.csv'  )
df.head()

C:\Users\Rowida Nagah


,image,top_left_x,top_left_y,bottom_right_x,bottom_right_y
0,user_3/A0.jpg,124,18,214,108
1,user_3/A1.jpg,124,18,214,108
2,user_3/A2.jpg,123,19,213,109
3,user_3/A3.jpg,122,21,212,111
4,user_3/A4.jpg,122,20,212,110


In [5]:
import cv2
import os
import glob
from skimage import data, io, filters
from skimage.transform import rescale
from skimage.color import rgb2gray
from skimage.feature import hog 
from skimage.transform import resize
img_dir = path + "\\EEG\\user_3" 
data_path = os.path.join(img_dir,'*g')
files = glob.glob(data_path)
def convertToGrayToHOG(imgVector):
    rgbImage = rgb2gray(imgVector)
    return hog(rgbImage)
from PIL import Image
os.chdir('EEG')
X = []
for i in range(len(df.image.values)):
    im = Image.open(r"{0}".format(df.image.values[i])).convert('LA')
    im1 = im.crop((df.top_left_x.values[i], df.top_left_y.values[i], df.bottom_right_x.values[i], df.bottom_right_y.values[i]))
    im1 = im1.resize((128, 128), Image.ANTIALIAS)
    im1 = hog(im1)
#print(len(im1))
    X.append(list(im1))


In [6]:
AY = 'ABCDEFGHIKLMNOPQRSTUVWXY'
Y = list()
for i in range(len(AY)):
    for j in range(10):
        Y.append(AY[i])
    

In [8]:
#data = pd.merge(DF , F)
DF = pd.DataFrame(X)
F = pd.DataFrame(Y)
data = pd.merge_asof(DF , F, left_index=True, right_index=True)

data.head()


,0_x,1,2,3,4,5,6,7,8,9,...,15867,15868,15869,15870,15871,15872,15873,15874,15875,0_y
0,0.207034,0.000000,0.008010,0.037995,0.067968,0.055907,0.040051,0.050661,0.000000,0.148184,...,0.000000,0.0,0.000000,0.000000,0.269353,0.000000,0.000000,0.000000,0.000000,A
1,0.210404,0.014370,0.133128,0.086218,0.128525,0.043109,0.018176,0.000000,0.079489,0.174552,...,0.000000,0.0,0.017927,0.040086,0.347031,0.056691,0.017927,0.000000,0.000000,A
2,0.224609,0.034242,0.032485,0.017121,0.068910,0.017121,0.043313,0.147452,0.024213,0.114850,...,0.144671,0.0,0.034099,0.049708,0.261024,0.026958,0.051149,0.053916,0.038124,A
3,0.218314,0.000000,0.042671,0.016867,0.158410,0.016867,0.042671,0.000000,0.000000,0.174720,...,0.086418,0.0,0.061107,0.000000,0.286187,0.000000,0.061107,0.000000,0.000000,A
4,0.112670,0.017995,0.113813,0.017995,0.223678,0.017995,0.034144,0.000000,0.025449,0.223678,...,0.226324,0.0,0.021338,0.000000,0.075441,0.000000,0.000000,0.000000,0.000000,A


In [11]:
X = data.iloc[:, :-1]
X.shape

(240, 15876)

In [16]:
Y = data.iloc[: , -1]
len(Y)

240

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state=0)


In [18]:
from sklearn.svm import SVC
svcmodel = SVC(kernel='linear', C=0.9, probability=True) 
svcmodel.fit(X_train, y_train) 

SVC(C=0.9, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [19]:
print(svcmodel.predict(X_test))

['L' 'H' 'D' 'H' 'L' 'X' 'O' 'X' 'X' 'U' 'H' 'V' 'I' 'O' 'Y' 'O' 'T' 'N'
 'X' 'A' 'L' 'S' 'I' 'N' 'A' 'C' 'U' 'K' 'B' 'R' 'F' 'E' 'P' 'N' 'P' 'M'
 'P' 'S' 'G' 'K' 'Q' 'E' 'V' 'B' 'O' 'K' 'L' 'A' 'M' 'Y' 'G' 'Q' 'Y' 'O'
 'K' 'D' 'Y' 'X' 'N' 'R' 'H' 'V' 'K' 'Y' 'V' 'O' 'K' 'W' 'N' 'H' 'A' 'L'
 'G' 'W' 'F' 'G' 'C' 'N' 'U' 'W']


In [21]:
from sklearn import metrics
y_pred = svcmodel.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred) )

Accuracy: 0.825


In [29]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train, y_train)
Yy_pred = neigh.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred) )

Accuracy: 0.825


In [23]:
df2 = pd.read_csv(path + '\\EEG\\user_4\\user_4_loc.csv'  )
df2.head()
X2 = []
from PIL import Image
for i in range(len(df.image.values)):
    im2 = Image.open(r"{0}".format(df2.image.values[i])).convert('LA')
    im2 = im2.crop((df2.top_left_x.values[i], df2.top_left_y.values[i], df2.bottom_right_x.values[i], df2.bottom_right_y.values[i]))
    im2 = im2.resize((128, 128), Image.ANTIALIAS)
    im2 = hog(im2)
#print(len(im1))
    X2.append(list(im2))

In [24]:
DF = pd.DataFrame(X2)


,0,1,2,3,4,5,6,7,8,9,...,15866,15867,15868,15869,15870,15871,15872,15873,15874,15875
0,0.222629,0.021505,0.068003,0.061157,0.153874,0.043009,0.027201,0.021505,0.0,0.222629,...,0.0,0.000000,0.0,0.000000,0.000000,0.276285,0.000000,0.000000,0.0,0.0
1,0.228478,0.019733,0.124800,0.075851,0.123546,0.084024,0.037440,0.019733,0.0,0.228478,...,0.0,0.074251,0.0,0.078755,0.000000,0.204191,0.000000,0.078755,0.0,0.0
2,0.229646,0.000000,0.073811,0.023341,0.083508,0.000000,0.088573,0.046682,0.0,0.147832,...,0.0,0.075116,0.0,0.000000,0.000000,0.325315,0.101376,0.000000,0.0,0.0
3,0.247232,0.061405,0.000000,0.000000,0.220746,0.047010,0.133792,0.047010,0.0,0.247232,...,0.0,0.000000,0.0,0.000000,0.000000,0.258613,0.000000,0.000000,0.0,0.0
4,0.232968,0.000000,0.104042,0.000000,0.110353,0.000000,0.086702,0.000000,0.0,0.249668,...,0.0,0.017184,0.0,0.048603,0.025616,0.114557,0.012808,0.032402,0.0,0.0


In [33]:
neigh.fit(X_train, y_train)
Yy_pred = neigh.predict(DF)
Yy_pred

array(['E', 'E', 'I', 'E', 'I', 'I', 'S', 'E', 'A', 'A', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'O', 'O', 'C', 'O', 'O', 'C',
       'C', 'C', 'C', 'O', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'E',
       'D', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'B', 'B',
       'B', 'B', 'V', 'F', 'F', 'F', 'C', 'F', 'G', 'G', 'G', 'G', 'G',
       'G', 'G', 'G', 'G', 'G', 'G', 'H', 'D', 'D', 'G', 'G', 'H', 'P',
       'G', 'P', 'Y', 'I', 'Y', 'I', 'E', 'I', 'I', 'I', 'I', 'E', 'D',
       'K', 'K', 'D', 'V', 'D', 'D', 'D', 'D', 'D', 'V', 'U', 'U', 'V',
       'U', 'K', 'K', 'U', 'K', 'U', 'E', 'E', 'D', 'M', 'E', 'E', 'E',
       'M', 'N', 'E', 'E', 'K', 'U', 'K', 'E', 'E', 'E', 'E', 'N', 'N',
       'O', 'O', 'O', 'O', 'O', 'O', 'C', 'O', 'C', 'C', 'H', 'H', 'H',
       'H', 'P', 'P', 'P', 'P', 'P', 'P', 'U', 'K', 'V', 'X', 'B', 'Q',
       'B', 'S', 'X', 'Q', 'K', 'K', 'U', 'R', 'U', 'K', 'B', 'B', 'B',
       'B', 'N', 'S', 'S', 'S', 'S', 'E', 'N', 'A', 'N', 'A', 'T

In [34]:
len(Yy_pred)

240

In [35]:
Y2 = list()
for i in range(len(AY)):
    for j in range(10):
        Y2.append(AY[i])
print("Accuracy:",metrics.accuracy_score(Y2, Yy_pred) )

Accuracy: 0.4375


In [36]:
YY_pred = svcmodel.predict(DF)
print("Accuracy:",metrics.accuracy_score(Y2, YY_pred) )

Accuracy: 0.48333333333333334


In [37]:
df3 = pd.read_csv(path + '\\EEG\\user_5\\user_5_loc.csv'  )
df3.head()
X3 = []
from PIL import Image
for i in range(len(df.image.values)):
    im3 = Image.open(r"{0}".format(df3.image.values[i])).convert('LA')
    im3 = im3.crop((df3.top_left_x.values[i], df3.top_left_y.values[i], df3.bottom_right_x.values[i], df3.bottom_right_y.values[i]))
    im3 = im3.resize((128, 128), Image.ANTIALIAS)
    im3 = hog(im3)
#print(len(im1))
    X3.append(list(im2))

In [41]:
DF3 = pd.DataFrame(X3)


In [43]:
pred = neigh.predict(DF3)
print("Accuracy:",metrics.accuracy_score(Y2,pred) )

Accuracy: 0.041666666666666664
